In [42]:
# -*- coding: utf-8 -*-
"""
Created on Tue Aug  8 15:37:10 2023

@author: Maybin Michael
"""

#packages
import pandas as pd
import numpy as np
import os
import traceback
import re
import plotly
import plotly.figure_factory as ff 
import plotly.express as px

class Data_Analytics():
    """
    Class to perform data analytics
    
    Attributes
    ----------
    df : Pandas DataFrame
        Data uploaded by the user
    cont_var : list
        List of continuous variables in the df dataframe
    categ_var : list
        List of categorical variables in the df dataframe
        
    Methods
    -------
        Details of the method are given with each method
    """
    def __init__(self):
        """
        Constructs/define all the necessary attributes for a use case
        
        Parameters
        ----------
        
        """
        self.df = pd.DataFrame()
        self.column_types = pd.DataFrame()
        
        
    def return_status(self,
                     error_code, 
                     status_msg = None,
                     error_trace = None,
                     output = None 
                     ):
        """
        Creates return dictionary for all the functions
        
        Parameters
        ----------
        error_code : int
            0 - Fucntion executed successfully, any other value -  execution failed
        status_msg : str, optional
            Status message of the function execution. If function failed, specify the reason.
            Default: None, 
        error_trace : str, optional
            Contains traceback of the error.
            Default : None
        output = any type
            Return value of the function if any
            Default : None
            
        Returns
        -------
        dictionary
            Output dictionary with multiple content
        """
        
        d = {"error_code": error_code,
             "status_msg": status_msg,
             "error_trace": error_trace,
             "output": output
             }
        return d

            
    def read_document(self, document_path):
        """
        Function to read the document uploaded by the user.

        Parameters
        ----------
        document_path : string
            Path of the document uploaded by the user

        Returns
        -------
        Dictionary
            Dictionary contains Dataframe with date uploaded by user.

        """
        try:
            self.df = pd.read_excel(document_path)
            self.cont_var = self.df.select_dtypes(exclude = ['object', 'datetime64']).columns
            self.categ_var = self.df.select_dtypes(include = ['object'], exclude = ['datetime64']).columns
            status_msg = "Function executed successfully"
            return self.return_status(0, status_msg)
        except:
            status_msg = "Upload file with .xlsx, .xls format"
            error_trace = ''.join(traceback.format_exc())
            return self.return_status(101, status_msg, error_trace)
        
    # Function to check data missingness, foreign values        
    def check_data_integrity(self):
        """
        This function checks data missingness columnwise
        Returns
        -------
        Dictionary
            Contains details of missing data

        """
        # checking data missingness columnwise
        try:
            data_missingness = self.df.isnull().sum()
            return self.return_status(0, output=data_missingness)
        except: 
            status_msg = "Data missingness could not be performed"
            error_trace = ''.join(traceback.format_exc())
            return self.return_status(102, status_msg, error_trace)
    
    # Function to remove rows with missing data and foreign values and to perform imputation
    def keep_data_integrity(self,
                            remove_missing_rows_continuous = False, 
                            remove_missing_rows_categorical = False, 
                            remove_foreign_rows= False, 
                            impute_missing_rows_continuous= 'mean',
                            impute_missing_rows_categorical= 'mode'
                            ):
        """
        Method to remove rows with missing data and foreign values and to perform imputation
        
        Parameters
        ----------
        remove_missing_rows_continuous : boolean, optional
            Removes missing rows in the case of continuous variables if True. The default is False.
        remove_missing_rows_categorical : boolean, optional
            Removes missing rows in the case of categorical variables if True. The default is False.
        remove_foreign_rows : boolean, optional
            Removes foreign values if True. The default is False.
        impute_missing_rows_continuous : str, optional
            Imputation is performed for continuous columns if any value is selected. The default is 'mean'. 
            mean - imputes missing values with mean. 
            meadian - impute missing values with median
            other methods are not supported now     
        impute_missing_rows_categorical : str, optional
            Imputation is performed for categorical columns if any value is selected. The default is 'mode'.
            mode - imputes missing values with mode
            other methods are not supported now

        Returns
        -------
        Dictionary
            DESCRIPTION.

        """
        
        # Remove/impute rows with missing values
        try:
            if remove_missing_rows_continuous == True:
                self.df = self.df[self.cont_var].dropna(axis=0, how='any')
            else:
                # ------------ impute with mean and median
                if impute_missing_rows_continuous == 'mean':
                    for i in self.cont_var:
                        self.df[i].fillna(value=self.df[i].mean(), inplace=True)
                elif impute_missing_rows_continuous == 'median':
                    for i in self.cont_var:
                        self.df[i].fillna(value=self.df[i].median(), inplace=True)
                else:
                    pass
                
            if remove_missing_rows_categorical == True:
                self.df = self.df[self.categ_var].dropna(axis=0, how='any')
            else:
                # impute with mode
                if impute_missing_rows_categorical == 'mode':
                    for i in self.categ_var:
                        self.df[i].fillna(value=self.df[i].mode()[0], inplace=True)
            
        except:
            status_msg = "Removal of rows with missing data or imputation could not be performed"
            error_trace = ''.join(traceback.format_exc())
            return self.return_status(104, status_msg, error_trace)
        
        # Remove/impute rows with foreign values
        try:
            values_count_index =[]
            for col in self.categ_var:
                l = self.df[col].value_counts().index.tolist()
                values_count_index.extend(l)
            values_count_index = set(values_count_index)
            foreign_values = [val for val in values_count_index if type(val)==str and not re.findall('\w+', val)]
            self.df = self.df.replace(foreign_values, np.nan)

            if remove_foreign_rows == True:
                # remove foreign rows
                self.df = self.df.dropna(axis=0)
            else:
                # perform imputation for categorical  features
                for i in self.categ_var:
                    self.df[i].fillna(value=self.df[i].mode()[0], inplace=True)
            
        except:
            status_msg = "Removal of rows with foreign data or imputation could not be performed"
            error_trace = ''.join(traceback.format_exc())
            return self.return_status(105, status_msg, error_trace)
        
        # Return updated df after resetting categ and cont var lists
        self.cont_var = self.df.select_dtypes(exclude = ['object', 'datetime64']).columns
        self.categ_var = self.df.select_dtypes(include = ['object'], exclude = ['datetime64']).columns
        return self.return_status(0, output=self.df)
    
    def describe_data(self):
        """
        Performs Descriptive analytics  

        Returns
        -------
        Dictionary
            DESCRIPTION.
        """
        
        try:
            # Summary of continuous type data
            self.cont_data_summary = pd.DataFrame(data = None, index = self.cont_var)
            self.cont_data_summary = self.cont_data_summary.assign(Mean = self.df[self.cont_var].mean())
            self.cont_data_summary = self.cont_data_summary.assign(Standard_Deviation = self.df[self.cont_var].std())
            self.cont_data_summary = self.cont_data_summary.assign(Variance = self.df[self.cont_var].var())
            self.cont_data_summary = self.cont_data_summary.assign(Min = self.df[self.cont_var].min())
            self.cont_data_summary = self.cont_data_summary.assign(Max = self.df[self.cont_var].max())
            self.cont_data_summary = self.cont_data_summary.assign(First_Quartile = self.df[self.cont_var].quantile(0.25))
            self.cont_data_summary = self.cont_data_summary.assign(Median = self.df[self.cont_var].median())
            self.cont_data_summary = self.cont_data_summary.assign(Third_Quartile = self.df[self.cont_var].quantile(0.75))
            
            # Summary of categorical data
            self.categ_data_summary = pd.DataFrame(data = None, index = self.categ_var)
            self.categ_data_summary = self.categ_data_summary.assign(Count = self.df[self.categ_var].count())
            self.categ_data_summary = self.categ_data_summary.assign(Count_of_Unique_Values = self.df[self.categ_var].nunique(axis=0))
            self.categ_data_summary = self.categ_data_summary.assign(Mode = self.df[self.categ_var].mode(axis=0).transpose())
            
            output = {"cont_data_summary": self.cont_data_summary,
                      "categ_data_summary": self.categ_data_summary
                      }
            return self.return_status(0, output=output)
        except:
            status_msg = "Descriptive statistics of data could not be performed"
            error_trace = ''.join(traceback.format_exc())
            return self.return_status(106, status_msg, error_trace)
        
    def show_distribution(self, feature):
        """
        This function returns distribution plot

        Parameters
        ----------
        feature : str
            Pass the feature name

        Returns
        -------
        image of the distribution of the feature

        """
        try:
            hist_data = [self.df[feature]]
            group_labels = [feature]
            self.fig =  ff.create_distplot(hist_data, group_labels)
            return self.return_status(0, output=self.fig)
        except:
            status_msg = "Plotting of distribution could not be performed"
            error_trace = ''.join(traceback.format_exc())
            return self.return_status(110, status_msg, error_trace)
            
    def show_trend(self, x_feature, y_feature):
        """
        

        Parameters
        ----------
        x_feature : str
            Feature to be set as x-axis
        y_feature : str
            Feature to be set as y-axis.

        Returns
        -------
        image
            Trendline image

        """
        try:
            self.fig = px.scatter(self.df, x=x_feature, y=y_feature, trendline="ols")
            return self.return_status(0, output=self.fig)
        except:
            status_msg = "Plotting of trendline could not be performed"
            error_trace = ''.join(traceback.format_exc())
            return self.return_status(103, status_msg, error_trace)
        
    def show_outliers(self, feature):
        """
        Performs data engineering - Plotting outliers using box and whisker method

        Returns
        -------
        image
            Box and whisker image

        """
        try:
            # plotting outliers using box and whisker 
            self.fig = px.histogram(self.df, x=feature, marginal='box', hover_data=self.df.columns)
            return self.return_status(0, output=self.fig)
        except:
            status_msg = "Plotting of outliers could not be performed"
            error_trace = ''.join(traceback.format_exc())
            return self.return_status(107, status_msg, error_trace)
        
    def outlier_removal_imputation(self,
                                remove_outliers = True, 
                                imputation_method = [],
                                selected_columns = [], 
                                lower_extreme = [], 
                                higher_extreme = [],
                                ):
        """
        Performs Data engineering - remove outliers using box and whisker method or Extreme value capping

        Parameters
        ----------
        remove_outliers : boolean, optional
            Removes outliers if True. The default is True. False - Do not remove outliers                                
        imputation_method : list, optional
            List of imputation method. The default is [] 
            mean - imputes outlier with mean 
            meadian - impute  outlier with median
            other methods are not supported now                                
        selected_columns : list, optional
            List of columns which should undergo outlier removal. The default is []
        lower_extreme : list, optional
            List of lower extreme values of respective columns. The default is [].
        higher_extreme : list, optional
            List of higher extreme values of respective columns. The default is [].

        Returns
        -------
        TYPE
            DESCRIPTION.

        """
        try:
            if remove_outliers == True:
                # remove outliers using box and whisker
                f = 1.3 # factor deciding whisker length
                i=0
                for k in selected_columns:
                    #calculating 1st and 3rd quantile in q
                    q = self.df[k].quantile([0.25, 0.75])
                    #Calculating the whisker's Min and Max value in d
                    d = list([q.iloc[0] - (f*(q.iloc[1]-q.iloc[0])), q.iloc[1] + (f*(q.iloc[1] - q.iloc[0]))])
                    # replace the outlier values by nan
                    self.df.loc[(self.df[k] < d[0]) | (self.df[k] > d[1]), k] = np.nan
                  
                    # Perform imputation
                    if imputation_method != []: 
                        if imputation_method[i] == 'mean':
                            self.df[k].fillna(value=self.df[k].mean(), inplace=True)
                        elif imputation_method[i] == 'median':
                            self.df[k].fillna(value=self.df[k].median(), inplace=True)
                        i+=1
                    else:
                        #perform extreme value capping
                        pass
            
        except:
            status_msg = "Outlier removal or imputation could not be performed"
            error_trace = ''.join(traceback.format_exc())
            return self.return_status(109, status_msg, error_trace)
    
    def download_file(self, folder_path):
        """
        Downloads the data file

        Parameters
        ----------
        folder_path : str
            Folder path where the file to be downloaded

        Returns
        -------
        DataFrame
            Dataframe which is converted and downloaded to the path

        """
        try:
            # Ensure output folder path exist
            if not os.path.isdir(folder_path):
                os.makedirs(folder_path)
            file_path = os.path.join(folder_path, 'file.xlsx')    
            self.df.to_excel(file_path)
            df_copy = self.df
            return self.return_status(0, output=df_copy)
        except:
            status_msg = "Downloading of the file could not be performed"
            error_trace = ''.join(traceback.format_exc())
            return self.return_status(108, status_msg, error_trace)
        
    def download_fig(self, folder_path):
        """
        Downloads the latest generated figure as html

        Parameters
        ----------
        folder_path : str
            Folder path where the figure to be downloaded

        Returns
        -------
        
        """
        try:
            # Ensure output folder path exist
            if not os.path.isdir(folder_path):
                os.makedirs(folder_path)
            file_path = os.path.join(folder_path, 'fig.html')    
            plotly.offline.plot(self.fig, filename=file_path)
            return self.return_status(0)
        except:
            status_msg = "Downloading of the figure could not be performed"
            error_trace = ''.join(traceback.format_exc())
            return self.return_status(111, status_msg, error_trace)

In [43]:
#from analytics_web_application import Data_Analytics
user1 = Data_Analytics()

o1 = user1.read_document("BostonHousePricing_excel.xlsx")
o1

{'error_code': 0,
 'status_msg': 'Function executed successfully',
 'error_trace': None,
 'output': None}

In [44]:
user1.df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,Country,Day
0,0.00632,18.0,NaN,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0,US,Sun
1,0.02731,0.0,NaN,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6,###,Mon
2,0.02729,100.0,NaN,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,?,Tue
3,0.03237,60.0,NaN,0,0.458,6.998,120.0,6.0622,3,222,18.7,394.63,2.94,33.4,…,Wed
4,0.06905,80.0,NaN,0,0.458,7.147,121.0,6.0622,3,222,18.7,396.90,5.33,36.2,India,NaN


In [45]:
o2 = user1.check_data_integrity()
o2

{'error_code': 0,
 'status_msg': None,
 'error_trace': None,
 'output': CRIM       0
 ZN         0
 INDUS      5
 CHAS       0
 NOX        0
 RM         0
 AGE        0
 DIS        0
 RAD        0
 TAX        0
 PTRATIO    0
 B          0
 LSTAT      0
 MEDV       0
 Country    9
 Day        6
 dtype: int64}

In [46]:
o3 = user1.keep_data_integrity(
                            remove_missing_rows_continuous = False, 
                            remove_missing_rows_categorical = False, 
                            remove_foreign_rows= False, 
                            impute_missing_rows_continuous= 'mean',
                            impute_missing_rows_categorical= 'mode'
                            )
user1.df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,Country,Day
0,0.00632,18.0,11.206387,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0,US,Sun
1,0.02731,0.0,11.206387,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6,US,Mon
2,0.02729,100.0,11.206387,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,US,Tue
3,0.03237,60.0,11.206387,0,0.458,6.998,120.0,6.0622,3,222,18.7,394.63,2.94,33.4,US,Wed
4,0.06905,80.0,11.206387,0,0.458,7.147,121.0,6.0622,3,222,18.7,396.90,5.33,36.2,India,Wed


In [47]:
o4 = user1.describe_data()
o4

{'error_code': 0,
 'status_msg': None,
 'error_trace': None,
 'output': {'cont_data_summary':                Mean  Standard_Deviation      Variance        Min       Max  \
  CRIM       3.613524            8.601545     73.986578    0.00632   88.9762   
  ZN        11.837945           23.927271    572.514280    0.00000  100.0000   
  INDUS     11.206387            6.820714     46.522136    0.46000   27.7400   
  CHAS       0.069170            0.253994      0.064513    0.00000    1.0000   
  NOX        0.554695            0.115878      0.013428    0.38500    0.8710   
  RM         6.284634            0.702617      0.493671    3.56100    8.7800   
  AGE       68.620751           28.631557    819.766083    2.90000  125.0000   
  DIS        3.795043            2.105710      4.434015    1.12960   12.1265   
  RAD        9.549407            8.707259     75.816366    1.00000   24.0000   
  TAX      408.237154          168.537116  28404.759488  187.00000  711.0000   
  PTRATIO   18.455534       

In [48]:
o4["output"]["cont_data_summary"]

,Mean,Standard_Deviation,Variance,Min,Max,First_Quartile,Median,Third_Quartile
CRIM,3.613524,8.601545,73.986578,0.00632,88.9762,0.082045,0.25651,3.677083
ZN,11.837945,23.927271,572.514280,0.00000,100.0000,0.000000,0.00000,16.250000
INDUS,11.206387,6.820714,46.522136,0.46000,27.7400,5.190000,9.79500,18.100000
CHAS,0.069170,0.253994,0.064513,0.00000,1.0000,0.000000,0.00000,0.000000
NOX,0.554695,0.115878,0.013428,0.38500,0.8710,0.449000,0.53800,0.624000
RM,6.284634,0.702617,0.493671,3.56100,8.7800,5.885500,6.20850,6.623500
AGE,68.620751,28.631557,819.766083,2.90000,125.0000,43.875000,77.70000,94.100000
DIS,3.795043,2.105710,4.434015,1.12960,12.1265,2.100175,3.20745,5.188425
RAD,9.549407,8.707259,75.816366,1.00000,24.0000,4.000000,5.00000,24.000000
TAX,408.237154,168.537116,28404.759488,187.00000,711.0000,279.000000,330.00000,666.000000


In [49]:
o4["output"]["categ_data_summary"]

,Count,Count_of_Unique_Values,Mode
Country,506,3,US
Day,506,5,Wed


In [50]:
user1.cont_var

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')

In [51]:
user1.categ_var

Index(['Country', 'Day'], dtype='object')

In [52]:
o5 = user1.show_distribution("AGE")
#o5["error_code"]
o5["output"]

In [53]:
o6=user1.show_trend(x_feature="AGE", y_feature="MEDV")
o6["output"]

In [54]:
o7=user1.show_outliers("AGE")
o7["output"]

In [55]:
o8 = user1.outlier_removal_imputation(
                                remove_outliers = True, 
                                imputation_method = ['mean','mean'],
                                selected_columns = ['ZN', 'AGE'], 
                                lower_extreme = [], 
                                higher_extreme = [],
                                )
user1.df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,Country,Day
0,0.00632,18.000000,11.206387,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0,US,Sun
1,0.02731,0.000000,11.206387,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6,US,Mon
2,0.02729,3.904494,11.206387,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,US,Tue
3,0.03237,3.904494,11.206387,0,0.458,6.998,120.0,6.0622,3,222,18.7,394.63,2.94,33.4,US,Wed
4,0.06905,3.904494,11.206387,0,0.458,7.147,121.0,6.0622,3,222,18.7,396.90,5.33,36.2,India,Wed


In [56]:
user1.download_file("downloads")

{'error_code': 0,
 'status_msg': None,
 'error_trace': None,
 'output':         CRIM         ZN      INDUS  CHAS    NOX     RM    AGE     DIS  RAD  \
 0    0.00632  18.000000  11.206387     0  0.538  6.575   65.2  4.0900    1   
 1    0.02731   0.000000  11.206387     0  0.469  6.421   78.9  4.9671    2   
 2    0.02729   3.904494  11.206387     0  0.469  7.185   61.1  4.9671    2   
 3    0.03237   3.904494  11.206387     0  0.458  6.998  120.0  6.0622    3   
 4    0.06905   3.904494  11.206387     0  0.458  7.147  121.0  6.0622    3   
 ..       ...        ...        ...   ...    ...    ...    ...     ...  ...   
 501  0.06263   0.000000  11.930000     0  0.573  6.593   69.1  2.4786    1   
 502  0.04527   0.000000  11.930000     0  0.573  6.120   76.7  2.2875    1   
 503  0.06076   0.000000  11.930000     0  0.573  6.976   91.0  2.1675    1   
 504  0.10959   0.000000  11.930000     0  0.573  6.794   89.3  2.3889    1   
 505  0.04741   0.000000  11.930000     0  0.573  6.030   80

In [57]:
user1.download_fig("downloads")

{'error_code': 0, 'status_msg': None, 'error_trace': None, 'output': None}